## Использование Spark для обучения и вывода на модели CatBoost 

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages ai.catboost:catboost-spark_3.3_2.12:1.1.1 pyspark-shell'

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Spark Catboost")
        .master("local[4]")
        .getOrCreate()
)

sc = spark.sparkContext

In [ ]:
import catboost_spark

## Набор данных для CatBoost 
объект Pool содержит строки со столбцами:
- features - вектор признаков
- label - целевая переменная
- cat_features - список категориальных признаков

и еще много разных полезных настроек
https://catboost.ai/en/docs/concepts/python-reference_pool

In [ ]:
from pyspark.sql import Row,SparkSession
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import *

In [ ]:
srcDataSchema = [
    StructField("features", VectorUDT()),
    StructField("label", StringType())
]

In [ ]:
# набор данных для обучения
trainData = [
    Row(Vectors.dense(0.1, 0.2, 0.11), "0"),
    Row(Vectors.dense(0.97, 0.82, 0.33), "1"),
    Row(Vectors.dense(0.13, 0.22, 0.23), "1"),
    Row(Vectors.dense(0.8, 0.62, 0.0), "0")
]

trainDf = (
    spark.createDataFrame(
        sc.parallelize(trainData),
        StructType(srcDataSchema))
)
trainPool = catboost_spark.Pool(trainDf)

In [ ]:
# тестовый набор данных
evalData = [
    Row(Vectors.dense(0.22, 0.33, 0.9), "1"),
    Row(Vectors.dense(0.11, 0.1, 0.21), "0"),
    Row(Vectors.dense(0.77, 0.0, 0.0), "1")
]
    
evalDf = spark.createDataFrame(spark.sparkContext.parallelize(evalData), StructType(srcDataSchema))
evalPool = catboost_spark.Pool(evalDf)

## Обучение классификатора

In [ ]:
classifier = catboost_spark.CatBoostClassifier()

In [ ]:
# обучение модел
model = classifier.fit(trainPool)

In [ ]:
# получение предсказаний
predictions = model.transform(evalPool.data)
predictions.show()

In [ ]:
# save the model
savedModelPath = "/tmp/binclass_model"
model.write().save(savedModelPath)

In [ ]:
# save the model as a local file in CatBoost native format
savedNativeModelPath = '/tmp/binclass_model.cbm'
model.saveNativeModel(savedNativeModelPath)

## Вывод на заранее обученой и сохраненной модели 
1. Создать (и указать, из какого файла брать данные модели) объект класса CatBoostClassificationModel
2. При вызове .transform будет создан Spark Job и выполнен на кластере, параллельно выполняя вывод

Можно проверить через Spark UI (:4040)

In [ ]:
# load the model (can be used in a different Spark session)

loadedModel = catboost_spark.CatBoostClassificationModel.load(savedModelPath)

predictionsFromLoadedModel = loadedModel.transform(evalPool.data)
predictionsFromLoadedModel.show()

In [ ]:
# load the model as a local file in CatBoost native format

loadedNativeModel = catboost_spark.CatBoostClassificationModel.loadNativeModel(savedNativeModelPath)

predictionsFromLoadedNativeModel = loadedNativeModel.transform(evalPool.data)
predictionsFromLoadedNativeModel.show()

## Явный перебор классифицируемых объектов на Spark RDD
1. Импортировать catboost (без _spark)
2. Создать CatBoostClassifier
3. Загрузить в него сохраненную модель
4. Создать из данных для классификации rdd методом parallelize
5. Вызвать model.predict для каждой строки rdd методом map
6. Запустить Job любым способом 

In [ ]:
import catboost

In [ ]:
model = catboost.CatBoostClassifier()
model.load_model(savedNativeModelPath)

In [ ]:
testData = [[0.22, 0.33, 0.9],[0.11, 0.1, 0.21],[0.77, 0.0, 0.0]]

test_rdd = spark.sparkContext.parallelize(testData)
results_rdd = test_rdd.map(lambda x: model.predict(x))
results = results_rdd.collect()
print(results)